In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-06 18:14:19.955612


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Collections

In [ ]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Parse Artists (Parallel)

In [3]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    arts.parseArtistModValFiles(modVal, force=force)
    #arts.createArtistModValMetadata(modVal=modVal)
    #arts.createArtistAlbumModValMetadata(modVal=modVal)
    
def parseArtistsParallel(nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    #result = pool.map_async(parseArtists, [35,34,33,32,31,30,29,28,27])

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#arts.parseArtistModValFiles(27, force=True)
arts.parseArtistModValFiles(29, force=True)

In [4]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
parseArtistsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Mon Jan 06, 2020 18:14:29 for Parsing All Artists
Current Time is Mon Jan 06, 2020 18:14:29 for Parallel Artist Parsing
Parsing Artist Files For ModVal 18
Parsing Artist Files For ModVal 0
Parsing Artist Files For ModVal 9
  ===> Found 6442 files. There are 8 new files to parse.
  ===> Found 6352 files. There are 6 new files to parse.
  ===> Found 6235 files. There are 8 new files to parse.
Saving 6 new artist IDs to /Volumes/Music/Discog/artists-db/9-DB.p
Saving 8 new artist IDs to /Volumes/Music/Discog/artists-db/0-DB.p
Saving 8 new artist IDs to /Volumes/Music/Discog/artists-db/18-DB.p
  --> This file is 11.1MB.
Saving 6352 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/9-Metadata.p
  --> This file is 377.7kB.
Saving 6352 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/9-MediaMetadata.p
  --> This file is 11.2MB.
Saving 6442 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/18-Metadata.p
  --> This fil

Parsing Artist Files For ModVal 6
Saving 6 new artist IDs to /Volumes/Music/Discog/artists-db/15-DB.p
  ===> Found 6311 files. There are 5 new files to parse.
Saving 9 new artist IDs to /Volumes/Music/Discog/artists-db/24-DB.p
Saving 5 new artist IDs to /Volumes/Music/Discog/artists-db/6-DB.p
  --> This file is 11.6MB.
Saving 6406 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/15-Metadata.p
  --> This file is 376.4kB.
Saving 6406 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/15-MediaMetadata.p
  --> This file is 3.8MB.
  --> This file is 10.8MB.
Saving 6290 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/24-Metadata.p
Parsing Artist Files For ModVal 16
  --> This file is 369.6kB.
Saving 6290 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/24-MediaMetadata.p
  --> This file is 3.5MB.
Parsing Artist Files For ModVal 25
  ===> Found 6319 files. There are 3 new files to parse.
  ===> Found 6399 fil

  --> This file is 363.9kB.
Saving 6322 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/30-MediaMetadata.p
  --> This file is 3.6MB.
Parsing Artist Files For ModVal 31
  ===> Found 6298 files. There are 8 new files to parse.
  --> This file is 12.2MB.
Saving 6268 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/39-Metadata.p
  --> This file is 366.9kB.
Saving 6268 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/39-MediaMetadata.p
Saving 8 new artist IDs to /Volumes/Music/Discog/artists-db/48-DB.p
  --> This file is 3.9MB.
Parsing Artist Files For ModVal 40
  ===> Found 6234 files. There are 5 new files to parse.
Saving 8 new artist IDs to /Volumes/Music/Discog/artists-db/31-DB.p
  --> This file is 10.5MB.
Saving 6499 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/48-Metadata.p
  --> This file is 374.5kB.
Saving 6499 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/48-MediaMet

Saving 12 new artist IDs to /Volumes/Music/Discog/artists-db/55-DB.p
  --> This file is 11.4MB.
Saving 6481 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/63-Metadata.p
  --> This file is 386.8kB.
Saving 6481 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/63-MediaMetadata.p
  --> This file is 3.7MB.
Parsing Artist Files For ModVal 64
  --> This file is 11.6MB.
Saving 6298 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/72-Metadata.p
  --> This file is 372.9kB.
Saving 6298 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/72-MediaMetadata.p
  ===> Found 6460 files. There are 7 new files to parse.
  --> This file is 3.7MB.
Parsing Artist Files For ModVal 73
  ===> Found 6313 files. There are 15 new files to parse.
  --> This file is 11.8MB.
Saving 6394 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/55-Metadata.p
  --> This file is 383.7kB.
Saving 6394 new artist IDs media data

  --> This file is 4.0MB.
Parsing Artist Files For ModVal 70
  --> This file is 11.9MB.
Saving 6512 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/78-Metadata.p
  --> This file is 377.4kB.
Saving 6512 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/78-MediaMetadata.p
  ===> Found 6300 files. There are 11 new files to parse.
  --> This file is 3.9MB.
Parsing Artist Files For ModVal 79
  --> This file is 13.0MB.
Saving 6347 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/61-Metadata.p
  --> This file is 370.9kB.
Saving 6347 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/61-MediaMetadata.p
  ===> Found 6360 files. There are 6 new files to parse.
  --> This file is 4.1MB.
Parsing Artist Files For ModVal 62
  ===> Found 6421 files. There are 7 new files to parse.
Saving 11 new artist IDs to /Volumes/Music/Discog/artists-db/70-DB.p
Saving 6 new artist IDs to /Volumes/Music/Discog/artists-db/79-DB.p
Sa

Saving 16 new artist IDs to /Volumes/Music/Discog/artists-db/95-DB.p
Saving 7 new artist IDs to /Volumes/Music/Discog/artists-db/87-DB.p
  --> This file is 11.1MB.
Saving 6498 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/95-Metadata.p
  --> This file is 389.3kB.
Saving 6498 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/95-MediaMetadata.p
  --> This file is 3.6MB.
Parsing Artist Files For ModVal 96
  ===> Found 6342 files. There are 2 new files to parse.
  --> This file is 11.5MB.
Saving 6356 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/87-Metadata.p
  --> This file is 375.9kB.
Saving 6356 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/87-MediaMetadata.p
  --> This file is 3.7MB.
Parsing Artist Files For ModVal 88
  ===> Found 6269 files. There are 5 new files to parse.
Saving 2 new artist IDs to /Volumes/Music/Discog/artists-db/96-DB.p
Saving 5 new artist IDs to /Volumes/Music/Discog/arti

# Albums

In [5]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


In [6]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [7]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Jan 06, 2020 19:49:25 for Parsing All Albums
Current Time is Mon Jan 06, 2020 19:49:25 for Parallel Album Parsing
Current Time is Mon Jan 06, 2020 19:49:25 for Parsing Album Files For ModVal NAN
Current Time is Mon Jan 06, 2020 19:49:25 for Parsing Album Files For ModVal 8
  Looking for artist albums in /Volumes/Music/Discog/albums/NAN
Current Time is Mon Jan 06, 2020 19:49:25 for Parsing Album Files For ModVal 17
  Looking for artist albums in /Volumes/Music/Discog/albums/17
  Looking for artist albums in /Volumes/Music/Discog/albums/8
Current Time is Mon Jan 06, 2020 19:49:25 for Processing ['NAN', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73

    Found 0   albums (4267 ) in 3000/6580	/Volumes/Music/Discog/albums/18/3117118
  --> This file is 15.5MB.
Saving 5228 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/10-ArtistMetadata.p
  --> This file is 1.1MB.
Saving 5228 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/10-ArtistAlbums.p
  --> This file is 2.6MB.
Current Time is Mon Jan 06, 2020 20:03:43 for Done with Parsing Album Files For ModVal 10
Process [{0}] took 5.6 minutes.
Current Time is Mon Jan 06, 2020 20:03:43 for Parsing Album Files For ModVal 11
  Looking for artist albums in /Volumes/Music/Discog/albums/11
  Found 6624 artist directories
    Found 1   albums (0    ) in 0/6624	/Volumes/Music/Discog/albums/11/1000811
    Found 0   albums (60   ) in 100/6624	/Volumes/Music/Discog/albums/11/1062111
    Found 0   albums (4796 ) in 3500/6580	/Volumes/Music/Discog/albums/18/3481518
    Found 1   albums (377  ) in 500/6624	/Volumes/Music/Discog/albums/11/133411
    Found 2   albums (5284 ) in 4000/658

    Found 1   albums (3601 ) in 2500/6668	/Volumes/Music/Discog/albums/12/2709012
    Found 0   albums (136  ) in 2000/4593	/Volumes/Music/Discog/albums/0/271000
  --> This file is 10.5MB.
Saving 3586 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/20-ArtistMetadata.p
  --> This file is 837.8kB.
Saving 3586 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/20-ArtistAlbums.p
  --> This file is 1.5MB.
Current Time is Mon Jan 06, 2020 20:24:45 for Done with Parsing Album Files For ModVal 20
Process [{0}] took 4.3 minutes.
Current Time is Mon Jan 06, 2020 20:24:45 for Parsing Album Files For ModVal 21
  Looking for artist albums in /Volumes/Music/Discog/albums/21
    Found 0   albums (187  ) in 2500/4593	/Volumes/Music/Discog/albums/0/3356300
  Found 3660 artist directories
    Found 0   albums (0    ) in 0/3660	/Volumes/Music/Discog/albums/21/100021
    Found 0   albums (9    ) in 100/3660	/Volumes/Music/Discog/albums/21/1065321
    Found 1   albums (239  ) in 3000/459

    Found 0   albums (9129 ) in 6000/6668	/Volumes/Music/Discog/albums/12/6551812
    Found 0   albums (61   ) in 1000/5916	/Volumes/Music/Discog/albums/23/1681023
    Found 0   albums (475  ) in 4000/4283	/Volumes/Music/Discog/albums/2/775802
    Found 0   albums (80   ) in 1500/5916	/Volumes/Music/Discog/albums/23/203723
    Found 0   albums (516  ) in 4282/4283	/Volumes/Music/Discog/albums/2/999502
Saving 516 (new) / 516 (parsed) / 18230 (total) album IDs to /Volumes/Music/Discog/albums-db/2-DB.p
    Found 0   albums (107  ) in 2000/5916	/Volumes/Music/Discog/albums/23/2415223
    Found 0   albums (133  ) in 2500/5916	/Volumes/Music/Discog/albums/23/28023
    Found 0   albums (164  ) in 3000/5916	/Volumes/Music/Discog/albums/23/3225923
    Found 0   albums (186  ) in 3500/5916	/Volumes/Music/Discog/albums/23/3640023
  --> This file is 12.8MB.
Saving 4283 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/2-ArtistMetadata.p
  --> This file is 950.5kB.
Saving 4283 new artist I

    Found 0   albums (107  ) in 1000/6194	/Volumes/Music/Discog/albums/25/1698525
    Found 0   albums (94   ) in 1000/6080	/Volumes/Music/Discog/albums/14/1702014
    Found 0   albums (3642 ) in 6500/6766	/Volumes/Music/Discog/albums/3/834003
    Found 0   albums (131  ) in 1500/6080	/Volumes/Music/Discog/albums/14/2061914
    Found 0   albums (168  ) in 1500/6194	/Volumes/Music/Discog/albums/25/2055725
    Found 0   albums (165  ) in 2000/6080	/Volumes/Music/Discog/albums/14/2462214
    Found 0   albums (213  ) in 2000/6194	/Volumes/Music/Discog/albums/25/2435925
    Found 0   albums (3854 ) in 6765/6766	/Volumes/Music/Discog/albums/3/99903
Saving 3854 (new) / 3854 (parsed) / 19328 (total) album IDs to /Volumes/Music/Discog/albums-db/3-DB.p
    Found 0   albums (204  ) in 2500/6080	/Volumes/Music/Discog/albums/14/2884614
    Found 0   albums (289  ) in 2500/6194	/Volumes/Music/Discog/albums/25/2820725
    Found 1   albums (231  ) in 3000/6080	/Volumes/Music/Discog/albums/14/331514
  

...

Process [{0}] took 4.2 minutes.
Current Time is Mon Jan 06, 2020 22:36:38 for Parsing Album Files For ModVal 90
  Looking for artist albums in /Volumes/Music/Discog/albums/90
    Found 0   albums (5    ) in 100/5082	/Volumes/Music/Discog/albums/81/1059581
    Found 0   albums (29   ) in 500/5082	/Volumes/Music/Discog/albums/81/13481
    Found 0   albums (13870) in 6632/6633	/Volumes/Music/Discog/albums/59/998559
Saving 13870 (new) / 13870 (parsed) / 27031 (total) album IDs to /Volumes/Music/Discog/albums-db/59-DB.p
    Found 2   albums (49   ) in 1000/5082	/Volumes/Music/Discog/albums/81/1681
  Found 6076 artist directories
    Found 0   albums (0    ) in 0/6076	/Volumes/Music/Discog/albums/90/1000590
    Found 0   albums (67   ) in 1500/5082	/Volumes/Music/Discog/albums/81/2017081
    Found 0   albums (12   ) in 100/6076	/Volumes/Music/Discog/albums/90/1077090
    Found 0   albums (97   ) in 2000/5082	/Volumes/Music/Discog/albums/81/2378581
    Found 1   albums (53   ) in 500/6076	/Vo

    Found 0   albums (15   ) in 100/4449	/Volumes/Music/Discog/albums/61/1080761
  --> This file is 9.7MB.
Saving 3642 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/82-ArtistMetadata.p
  --> This file is 843.4kB.
Saving 3642 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/82-ArtistAlbums.p
    Found 0   albums (60   ) in 500/4449	/Volumes/Music/Discog/albums/61/1451861
    Found 2   albums (516  ) in 4466/4467	/Volumes/Music/Discog/albums/91/999491
Saving 518 (new) / 518 (parsed) / 16376 (total) album IDs to /Volumes/Music/Discog/albums-db/91-DB.p
  --> This file is 1.4MB.
Current Time is Mon Jan 06, 2020 22:43:21 for Done with Parsing Album Files For ModVal 82
Process [{0}] took 3.3 minutes.
Current Time is Mon Jan 06, 2020 22:43:21 for Parsing Album Files For ModVal 83
  Looking for artist albums in /Volumes/Music/Discog/albums/83
    Found 0   albums (122  ) in 1000/4449	/Volumes/Music/Discog/albums/61/1917761
  Found 4445 artist directories
    Found 0   alb

  --> This file is 2.0MB.
Current Time is Mon Jan 06, 2020 22:52:19 for Done with Parsing Album Files For ModVal 98
Process [{0}] took 5.8 minutes.
Current Time is Mon Jan 06, 2020 22:52:19 for Parsing Album Files For ModVal 99
  Looking for artist albums in /Volumes/Music/Discog/albums/99
    Found 1   albums (3432 ) in 6429/6430	/Volumes/Music/Discog/albums/92/999192
Saving 3433 (new) / 3433 (parsed) / 17920 (total) album IDs to /Volumes/Music/Discog/albums-db/92-DB.p
  --> This file is 22.7MB.
Saving 6263 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/84-ArtistMetadata.p
  --> This file is 1.3MB.
Saving 6263 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/84-ArtistAlbums.p
  Found 4730 artist directories
    Found 0   albums (0    ) in 0/4730	/Volumes/Music/Discog/albums/99/100199
    Found 1   albums (5    ) in 100/4730	/Volumes/Music/Discog/albums/99/106999
  --> This file is 3.8MB.
Current Time is Mon Jan 06, 2020 22:52:54 for Done with Parsing Album Files 

    Found 0   albums (465  ) in 4500/4557	/Volumes/Music/Discog/albums/86/96186
    Found 0   albums (471  ) in 4556/4557	/Volumes/Music/Discog/albums/86/999186
Saving 471 (new) / 471 (parsed) / 15790 (total) album IDs to /Volumes/Music/Discog/albums-db/86-DB.p
    Found 0   albums (210  ) in 4500/6216	/Volumes/Music/Discog/albums/94/4775694
    Found 0   albums (223  ) in 5000/6216	/Volumes/Music/Discog/albums/94/5309494
    Found 0   albums (257  ) in 5500/6216	/Volumes/Music/Discog/albums/94/6406894
    Found 0   albums (295  ) in 6000/6216	/Volumes/Music/Discog/albums/94/85994
  --> This file is 12.1MB.
Saving 4557 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/86-ArtistMetadata.p
    Found 0   albums (307  ) in 6215/6216	/Volumes/Music/Discog/albums/94/9994
Saving 307 (new) / 307 (parsed) / 38110 (total) album IDs to /Volumes/Music/Discog/albums-db/94-DB.p
  --> This file is 991.2kB.
Saving 4557 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/86-ArtistAlbums

  --> This file is 2.5MB.
Current Time is Mon Jan 06, 2020 23:07:20 for Done with Parsing Album Files For ModVal 96
Process [{0}] took 3.0 minutes.
Current Time is Mon Jan 06, 2020 23:07:20 for Parsing Album Files For ModVal 97
  Looking for artist albums in /Volumes/Music/Discog/albums/97
  Found 4331 artist directories
    Found 0   albums (0    ) in 0/4331	/Volumes/Music/Discog/albums/97/1001097
    Found 1   albums (13   ) in 100/4331	/Volumes/Music/Discog/albums/97/1067297
    Found 0   albums (44   ) in 500/4331	/Volumes/Music/Discog/albums/97/1370997
    Found 0   albums (101  ) in 1000/4331	/Volumes/Music/Discog/albums/97/1841697
    Found 0   albums (160  ) in 1500/4331	/Volumes/Music/Discog/albums/97/2435297
    Found 0   albums (215  ) in 2000/4331	/Volumes/Music/Discog/albums/97/3021597
    Found 0   albums (281  ) in 2500/4331	/Volumes/Music/Discog/albums/97/3705997
    Found 0   albums (332  ) in 3000/4331	/Volumes/Music/Discog/albums/97/4573397
    Found 0   albums (374 